# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
S. Hannon  ->  S. Hannon  |  ['S. Hannon']
J. Li  ->  J. Li  |  ['J. Li']
J. Neumann  ->  J. Neumann  |  ['J. Neumann']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
J. Klevas  ->  J. Klevas  |  ['J. Klevas']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


J. Mah  ->  J. Mah  |  ['J. Mah']
Arxiv has 85 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2401.15142


extracting tarball to tmp_2401.15142...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2401.15142/paper.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'authors' from 'tmp_2401.15142/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'affils' from 'tmp_2401.15142/affils.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

 done.
Retrieving document from  https://arxiv.org/e-print/2401.16155


extracting tarball to tmp_2401.16155...

 done.


J. Mah  ->  J. Mah  |  ['J. Mah']


Found 93 bibliographic references in tmp_2401.16155/Andama_Mah_Bitsch_2023.bbl.
syntax error in line 515: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.15142-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.15142) | **PHANGS-JWST: Data Processing Pipeline and First Full Public Data Release**  |
|| T. G. Williams, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>S. Hannon</mark>, <mark>J. Li</mark>, <mark>J. Neumann</mark>, <mark>N. Neumayer</mark> |
|*Appeared on*| *2024-01-30*|
|*Comments*| *50 pages (27 in Appendices), 54 Figures (39 in Appendices), 2 Tables. Submitted to ApJS. Data available at this https URL*|
|**Abstract**| The exquisite angular resolution and sensitivity of JWST is opening a new window for our understanding of the Universe. In nearby galaxies, JWST observations are revolutionizing our understanding of the first phases of star formation and the dusty interstellar medium. Nineteen local galaxies spanning a range of properties and morphologies across the star-forming main sequence have been observed as part of the PHANGS-JWST Cycle 1 Treasury program at spatial scales of $\sim$5-50pc. Here, we describe pjpipe, an image processing pipeline developed for the PHANGS-JWST program that wraps around and extends the official JWST pipeline. We release this pipeline to the community as it contains a number of tools generally useful for JWST NIRCam and MIRI observations. Particularly for extended sources, pjpipe products provide significant improvements over mosaics from the MAST archive in terms of removing instrumental noise in NIRCam data, background flux matching, and calibration of relative and absolute astrometry. We show that slightly smoothing F2100W MIRI data to 0.9" (degrading the resolution by about 30 percent) reduces the noise by a factor of $\approx$3. We also present the first public release (DR1.0.1) of the pjpipe processed eight-band 2-21 $\mu$m imaging for all nineteen galaxies in the PHANGS-JWST Cycle 1 Treasury program. An additional 55 galaxies will soon follow from a new PHANGS-JWST Cycle 2 Treasury program. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.15172-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.15172) | **Entropy-calibrated stellar modeling: Testing and improving the use of  prescriptions for entropy of adiabatic convection**  |
|| L. Manchon, et al. -- incl., <mark>J. Klevas</mark> |
|*Appeared on*| *2024-01-30*|
|*Comments*| *18 pages, 11 figures. Submitted to Astronomy and Astrophysics, recommended for publication with minor revisions*|
|**Abstract**| The modeling of convection is a long standing problem in stellar physics. Up-to-now, all ad hoc models rely on a free parameter alpha (among others) which has no real physical justification and is therefore poorly constrained. However, a link exists between this free parameter and the entropy of the stellar adiabat. Prescriptions, derived from 3D stellar atmospheric models, are available that provide entropy as a function of stellar atmospheric parameters (effective temperature, surface gravity, chemical composition). This can provide constraints on alpha through the development of entropy-calibrated models. Several questions arise as these models are increasingly used. Which prescription should be used? How do uncertainties impact entropy-calibrated models? We aim to study the three existing prescriptions and determine which one should be used, and how. We implemented the entropy-calibration method into the stellar evolution code Cesam2k20 and performed comparisons with the Sun and the alpha Cen system. In addition, we used data from the CIFIST grid of 3D atmosphere models to evaluate the accuracy of the prescriptions. Of the three entropy prescriptions available, we determine which one best reproduces the entropies of the 3D models. We also demonstrate that the entropy obtained from this prescription should be corrected for the evolving chemical composition and for an entropy offset different between various EoS tables, following a precise procedure, otherwise classical parameters obtained from the models will be strongly biased. Finally, we also provide table with entropy of the adiabat of the CIFIST grid, as well as fits of these entropies. We performed a precise examination of entropy-calibrated modelling, and gave recommendations on which adiabatic entropy prescription to use, how to correct it and to implement the method into a stellar evolution code. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.16155-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.16155) | **What stars can form planets: planetesimal formation at low metallicity**  |
|| G. Andama, <mark>J. Mah</mark>, B. Bitsch |
|*Appeared on*| *2024-01-30*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**| The exoplanet diversity has been linked to the disc environment in which they form, where the host star metallicity and the formation pathways play a crucial role. In the context of the core accretion paradigm, the initial stages of planet formation require the growth of dust material from micrometre size to planetesimal size bodies before core accretion can kick in. Although numerous studies have been conducted on planetesimal formation, it is still poorly understood how this process takes place in low metallicity stellar environments. We explore planetesimals formation in stellar environments primarily with low metallicity. We performed global 1D viscous disc evolution simulations including grain growth, evaporation and condensation of chemical species at ice lines. We followed the formation of planetesimals during disc evolution and tested different metallicities, disc sizes and turbulent viscosity strengths. We find that at solar and sub-solar metallicities, there is a significant enhancement in the midplane dust-to-gas mass ratios at the ice lines but this leads to planetesimal formation only at the water ice line. In our simulations, [Fe/H] = -0.6 is the lowest limit of metallicity for planetesimal formation where a few Earth masses of planetesimals could form. For such extreme disc environments, large discs are more conducive than small discs for forming large amounts of planetesimals at a fixed metallicity, because the pebble flux can be maintained for a longer time resulting in a longer and more efficient planetesimal formation phase. At lower metallicities, planetesimal formation is less supported in quiescent discs compared to turbulent discs, because the pebble flux can be maintained for a longer time. The amount of planetesimals formed at sub-solar metallicities in our simulations places a limit on core sizes that could possibly only result in the formation of super-Earths. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.15149-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.15149) | **$ω$ Centauri: A MUSE discovery of a counter-rotating core**  |
|| R. Pechetti, et al. -- incl., <mark>N. Neumayer</mark> |
|*Appeared on*| *2024-01-30*|
|*Comments*| *Accepted for publication in MNRAS, 16 pages, 9 figures*|
|**Abstract**| $\omega$ Centauri is considered the most massive globular cluster of the Milky Way and likely the former nuclear star cluster of a galaxy accreted by the Milky Way. It is speculated to contain an intermediate-mass black hole (IMBH) from several dynamical models. However, uncertainties regarding the location of the cluster center or the retention of stellar remnants limit the robustness of the IMBH detections reported so far. In this paper, we derive and study the stellar kinematics from the highest-resolution spectroscopic data yet, using the Multi Unit Spectroscopic Explorer (MUSE) in the narrow field mode (NFM) and wide field mode (WFM). Our exceptional data near the center reveal for the first time that stars within the inner 20" ($\sim$0.5 pc) counter-rotate relative to the bulk rotation of the cluster. Using this dataset, we measure the rotation and line-of-sight velocity dispersion (LOSVD) profile out to 120$''$ with different centers proposed in the literature. We find that the velocity dispersion profiles using different centers match well with those previously published. Based on the counter--rotation, we determine a kinematic center and look for any signs of an IMBH using the high-velocity stars close to the center. We do not find any significant outliers $>$60 km/s within the central 20$''$, consistent with no IMBH being present at the center of $\omega$ Centauri. A detailed analysis of Jeans' modeling of the putative IMBH will be presented in the next paper of the series. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.15438-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.15438) | **Relations of rotation and chromospheric activity to stellar age for FGK  dwarfs from Kepler and LAMOST**  |
|| L. Ye, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-01-30*|
|*Comments*| **|
|**Abstract**| The empirical relations between rotation period, chromospheric activity, and age can be used to estimate stellar age. To calibrate these relations, we present a catalog, including the masses and ages of 52,321 FGK dwarfs, 47,489 chromospheric activity index $logR^{+}_{HK}$, 6,077 rotation period $P_{rot}$ and variability amplitude $S_{ph}$, based on data from LAMOST DR7, Kepler and Gaia DR3. We find a pronounced correlation among $P_{rot}$, age, and [Fe/H] throughout the main-sequence phase for F dwarfs. However, the decrease of $logR^{+}_{HK}$ over time is not significant except for those with [Fe/H] $<$ $-$0.1. For G dwarfs, both $P_{rot}$ and $logR^{+}_{HK}$ are reliable age probes in the ranges $\sim$ 2-11 Gyr and $\sim$ 2-13 Gyr, respectively. K dwarfs exhibit a prominent decrease in $logR^{+}_{HK}$ within the age range of $\sim$ 3-13 Gyr when the relation of $P_{rot}-\tau$ is invalid. These relations are very important for promptly estimating the age of a vast number of stars, thus serving as a powerful tool in advancing the fields of exoplanet properties, stellar evolution, and Galactic-archaeology. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2401.15142.md
    + _build/html/tmp_2401.15142/figs/mast_comparison/ngc0628_mast_comparison.png
    + _build/html/tmp_2401.15142/figs/mosaics/ngc0628_mosaics.png
    + _build/html/tmp_2401.15142/figs/early_science/ngc0628_early_science_comparison.png
exported in  _build/html/2401.15172.md
    + _build/html/tmp_2401.15172/./figures/horiz_plot_error_magic-cifist_3x2.png
    + _build/html/tmp_2401.15172/./figures/ttau32.png
    + _build/html/tmp_2401.15172/./figures/combined_acen_RL_alpha_dcz.png
exported in  _build/html/2401.16155.md
    + _build/html/tmp_2401.16155/./Figures/fig1_sigma_draz_rc100_alpha4.png
    + _build/html/tmp_2401.16155/./Figures/fig2_sigma_draz_rc100_alpha3.png
    + _build/html/tmp_2401.16155/./Figures/fig5_Mass_vs_t_rc_draz.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\onef}{1/f}$
$\newcommand{\pipename}{{\tt pjpipe}\xspace}$
$\newcommand{\Ox}{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\newcommand{\UGent}{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}$
$\newcommand{\STScI}{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\MPIA}{Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117, Heidelberg, Germany}$
$\newcommand{\AURA}{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\UCSD}{Department of Astronomy \& Astrophysics, University of California, San Diego, 9500 Gilman Dr., La Jolla, CA 92093, USA}$
$\newcommand{\JHU}{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\newcommand{\OSU}{Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, OH 43210, USA}$
$\newcommand{ÇAPP}{Center for Cosmology and Astroparticle Physics (CCAPP), 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\newcommand{\ARI}{Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstr. 12-14, D-69120 Heidelberg, Germany}$
$\newcommand{\UConn}{Department of Physics, University of Connecticut, 196A Auditorium Road, Storrs, CT 06269, USA}$
$\newcommand{\UHawaii}{Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}$
$\newcommand{\UniCA}{Université C\^{o}te d'Azur, Observatoire de la C\^{o}te d'Azur, CNRS, Laboratoire Lagrange, 06000, Nice, France}$
$\newcommand{\UAlberta}{Dept. of Physics, University of Alberta, 4-183 CCIS, Edmonton, Alberta, T6G 2E1, Canada}$
$\newcommand{\Arcetri}{INAF — Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50125, Florence, Italy}$
$\newcommand{\UWyoming}{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\newcommand{\LJMU}{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}$
$\newcommand{\ITA}{Universität Heidelberg, Zentrum für Astronomie, Institut für Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\newcommand{\CfA}{Center for Astrophysics \mid Harvard \& Smithsonian, 60 Garden St., 02138 Cambridge, MA, USA}$
$\newcommand{\MPE}{Max-Planck-Institut für Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\newcommand{\Surrey}{Department of Physics, University of Surrey, Guildford GU2 7XH, UK}$
$\newcommand{\ESO}{European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei München, Germany}$
$\newcommand{\IWR}{Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\newcommand{\ulyon}{Univ Lyon, Univ Lyon1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574, F-69230 Saint-Genis-Laval France}$
$\newcommand{\COOL}{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\newcommand{\OAN}{Observatorio Astron{ó}mico Nacional (IGN), C/ Alfonso XII 3, E-28014 Madrid, Spain}$
$\newcommand{\UBonn}{Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121 Bonn, Germany}$
$\newcommand{\kipac}{Kavli Institute for Particle Astrophysics \& Cosmology (KIPAC), Stanford University, CA 94305, USA}$
$\newcommand{\Umanc}{Jodrell Bank Centre for Astrophysics, Department of Physics and Astronomy, University of Manchester, Oxford Road, Manchester M13 9PL, UK}$
$\newcommand{\NRAO}{National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903, USA}$
$\newcommand{\ANU}{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{\AThreeD}{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}$
$\newcommand{\Whitman}{Whitman College, 345 Boyer Avenue, Walla Walla, WA 99362, USA}$
$\newcommand{\IAC}{Instituto de Astrofísica de Canarias, C/ Vía Láctea s/n, E-38205, La Laguna, Spain}$
$\newcommand{\ULL}{Departamento de Astrofísica, Universidad de La Laguna, Av. del Astrofísico Francisco Sánchez s/n, E-38206, La Laguna, Spain}$
$\newcommand{\Princeton}{Department of Astrophysical Sciences, Princeton University, 4 Ivy Lane, Princeton, NJ 08544, USA}$
$\newcommand{\IRAM}{IRAM, 300 rue de la Piscine, 38400 Saint Martin d'Héres, France}$
$\newcommand{\LERMA}{LERMA, Observatoire de Paris, PSL Research University, CNRS, Sorbonne Universités, 75014 Paris}$
$\newcommand{\YB}{Centro de Desarrollos Tecnológicos, Observatorio de Yebes (IGN), 19141 Yebes, Guadalajara, Spain}$</div>



<div id="title">

# PHANGS-$_ JWST_$: Data Processing Pipeline and First Full Public Data Release

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2401.15142-b31b1b.svg)](https://arxiv.org/abs/2401.15142)<mark>Appeared on: 2024-01-30</mark> -  _50 pages (27 in Appendices), 54 Figures (39 in Appendices), 2 Tables. Submitted to ApJS. Data available at this https URL_

</div>
<div id="authors">

T. G. Williams, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>S. Hannon</mark>, <mark>J. Li</mark>, <mark>J. Neumann</mark>, <mark>N. Neumayer</mark>

</div>
<div id="abstract">

**Abstract:** The exquisite angular resolution and sensitivity of $_ JWST_$ is opening a new window for our understanding of the Universe. In nearby galaxies, $_ JWST_$ observations are revolutionizing our understanding of the first phases of star formation and the dusty interstellar medium. Nineteen local galaxies spanning a range of properties and morphologies across the star-forming main sequence have been observed as part of the PHANGS- $_ JWST_$ Cycle 1 Treasury program at spatial scales of $\sim$ 5--50 pc.  Here, we describe $\pipename$ , an image processing pipeline developed for the PHANGS- $_ JWST_$ program that wraps around and extends the official $_ JWST_$ pipeline. We release this pipeline to the community as it contains a number of tools generally useful for $_ JWST_$ NIRCam and MIRI observations. Particularly for extended sources, $\pipename$ products provide significant improvements over mosaics from the MAST archive in terms of removing instrumental noise in NIRCam data, background flux matching, and calibration of relative and absolute astrometry. We show that slightly smoothing F2100W MIRI data to 0 $\farcs$ 9 (degrading the resolution by about 30 percent) reduces the noise by a factor of $\approx$ 3. We also present the first public release (DR1.0.1) of the $\pipename$ processed eight-band 2-21 $\mu$ m imaging for all nineteen galaxies in the PHANGS- $_ JWST_$ Cycle 1 Treasury program. An additional 55 galaxies will soon follow from a new PHANGS- $_ JWST_$ Cycle 2 Treasury program.

</div>

<div id="div_fig1">

<img src="tmp_2401.15142/figs/mast_comparison/ngc0628_mast_comparison.png" alt="Fig15" width="100%"/>

**Figure 15. -** Comparison between our reprocessing and Level Three MAST-obtained mosaics for NGC 628. The difference here is the MAST version subtracted from our reprocessed version, after reprojecting to the same WCS. As can be seen, the astrometry is somewhat different in NIRCam, and the striping in the MAST mosaic is clear. The background levels are substantially different in the MIRI bands, due to a combination of our improved background matching algorithm and the fact that the MAST mosaics do not include our dedicated background observations. (*fig:mast_comparison*)

</div>
<div id="div_fig2">

<img src="tmp_2401.15142/figs/mosaics/ngc0628_mosaics.png" alt="Fig10" width="100%"/>

**Figure 10. -** Final mosaics for each band for NGC 628. NIRCam filters are shown in the top row, and MIRI on the bottom. Each image is on a linear stretch between the $2^{\rm nd}$ and $98^{\rm th}$ percentiles of the pixel distribution, to highlight the fainter structure in the images. (*fig:final_mosaics*)

</div>
<div id="div_fig3">

<img src="tmp_2401.15142/figs/early_science/ngc0628_early_science_comparison.png" alt="Fig14" width="100%"/>

**Figure 14. -** Comparison between our latest reprocessing and our early science mosaic for NGC 628. The difference here is the early science version subtracted from our latest reprocessed version, after reprojecting to the same WCS. There are significant differences in the short NIRCam band, caused by both background level matching and our new destriping method. At long NIRCam wavelengths, the differences are relatively minor. For MIRI, the improved calibration shows up as a galaxy imprint (most clear at F770W). Differences in astrometry show neighboring red/blue shifts and the background matching algorithm shows clear steps between mosaic tiles. (*fig:early_sci_comparison*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2401.15142"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\corr}[1]{#1}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\clouis}[1]{{\bf\color{louis}#1}}$
$\newcommand{\cfederico}[1]{{\bf\color{federico}#1}}$
$\newcommand{\cmorgan}[1]{{\bf\color{morgan}#1}}$
$\newcommand{\cmjo}[1]{{\bf\color{mjo}#1}}$
$\newcommand{\caldo}[1]{{\bf\color{aldo}#1}}$
$\newcommand{\caldotwo}[1]{{\bf\color{aldotwo}#1}}$
$\newcommand{\cyveline}[1]{{\bf\color{yveline}#1}}$
$\newcommand{\chans}[1]{{\bf\color{hans}#1}}$
$\newcommand{\cjosefina}[1]{{\bf\color{josefina}#1}}$
$\newcommand{\claurent}[1]{{\bf\color{laurent}#1}}$
$\newcommand{\}{eos}$
$\newcommand{\}{amlt}$
$\newcommand{\}{afst}$
$\newcommand{\}{cesampp}$
$\newcommand{\}{stagger}$
$\newcommand{\}{cobold}$
$\newcommand{\}{acen}$
$\newcommand{\}{feh}$
$\newcommand{\}{sad}$
$\newcommand{\}{spresc}$
$\newcommand{\}{sbot}$
$\newcommand{\}{srhd}$
$\newcommand{\}{presc}$
$\newcommand{\}{ds}$
$\newcommand{\}{teff}$
$\newcommand{\}{logteff}$
$\newcommand{\}{logg}$
$\newcommand{\}{erg}$
$\newcommand{\}{K}$
$\newcommand{\}{sunit}$
$\newcommand{\}{punit}$
$\newcommand{\}{lunit}$
$\newcommand{\}{rhounit}$
$\newcommand{\}{gunit}$
$\newcommand{\}{Tunit}$
$\newcommand{\}{ttau}$</div>



<div id="title">

# Entropy-calibrated stellar modeling: Testing and improving the use of prescriptions for entropy of adiabatic convection

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2401.15172-b31b1b.svg)](https://arxiv.org/abs/2401.15172)<mark>Appeared on: 2024-01-30</mark> -  _18 pages, 11 figures. Submitted to Astronomy and Astrophysics, recommended for publication with minor revisions_

</div>
<div id="authors">

L. Manchon, et al. -- incl., <mark>J. Klevas</mark>

</div>
<div id="abstract">

**Abstract:** The modeling of convection is a long standing problem in stellar physics. Up-to-now, all _ad hoc_ models rely on a free parameter $\alpha$ (among others) which has no real physical justification and is therefore poorly constrained. However, a link exists between this free parameter and the entropy of the stellar adiabat. Prescriptions, derived from 3D stellar atmospheric models, are available that provide entropy as a function of stellar atmospheric parameters (effective temperature, surface gravity, chemical composition). This can provide constraints on $\alpha$ through the development of entropy-calibrated models. Several questions arise as these models are increasingly used.    Which prescription should be used? Should they be used in there original form? How do uncertainties impact entropy-calibrated models? We aim to study in detail the three existing prescriptions and determine which one should be used, and how. We implemented the entropy-calibration method into the stellar evolution code ( $\cesampp$ ) and performed comparisons with the Sun and the $\acen$ system. ${In addition, we used data from the CIFIST grid of 3D atmosphere models to evaluate the accuracy of the prescriptions.}$ Of three entropy prescriptions available, we determine which has the best functional form for reproducing the entropies of the 3D models.  However, the coefficients involved in this formulation must not be taken from the original paper, because they were calibrated against an improper set of entropies. We also demonstrate that the entropy obtained from this prescription should be corrected for the evolving chemical composition and for an entropy offset different between various EoS tables, following a precise procedure, otherwise classical parameters obtained from the models will be strongly biased. Finally, we also provide table with entropy of the adiabat of the CIFIST grid, as well as fits of these entropies. We performed a precise examination of entropy-calibrated modelling, and we gave recommendations on which adiabatic entropy prescription to use, how to correct it and how to implement the method into a stellar evolution code.

</div>

<div id="div_fig1">

<img src="tmp_2401.15172/./figures/horiz_plot_error_magic-cifist_3x2.png" alt="Fig8" width="100%"/>

**Figure 8. -** Specific entropy differences in the adiabat $\Delta s^\presc \equiv s_{\rm RHD} - s_{\rm presc}$ between values provided by the CIFIST grid  ([Ludwig, Caffau and Steffen 2009]()) , and values predicted by the M13 prescription, Eq. \eqref{eq:s_mag}. Left column: orignal coefficients from  ([Magic, Collet and Asplund 2013]())  are used. Here, entropy from the CIFIST grid has been corrected by an offset of $0.04794\cdot\sunit$ corresponding to the entropy difference between the solar atmosphere models in the CIFIST and \stagger grids. Right columns: Same as left column but the coefficients involved in the M13 mathematical form have been re-calibrated with the CIFIST entropies. Rows: Each row displays models with a given metallicity: $\feh = +0.0$(1\st row), $-1.0$(2\nd row) and $-2.0$(3\rd row). (*fig:error_magic*)

</div>
<div id="div_fig2">

<img src="tmp_2401.15172/./figures/ttau32.png" alt="Fig10" width="100%"/>

**Figure 10. -** Top left panel: Comparison of luminosity (solid lines) and effective temperature (dashed lines), for four models computed with different choices of $T(\tau)$ relation. The displayed quantity is the relative difference of $L$ or $\teff$, with respect to model computed with the Eddington $T(\tau)$ relation: $\delta X \equiv (X_{T(\tau)} - X_{\rm Edding.}) / X_{\rm Edding.}$. Top right panel: same as top left panel but, in the case where $X \equiv \sad$, we display the absolute difference. Indeed, $\sad$ being defined up to a constant, the relative difference would have no meaning. $\Delta s^\presc$ is in the units: $\sunit$. Bottom left panel: Evolution of $\amlt$ for different $T(\tau)$ relations: Eddington (blue), [Vernazza, Avrett and Loeser (1981)]()(green) and [ and Ball (2021)]()(red). Bottom right panel: Temperature as a function of the pressure for the same four final models, at solar age. (*fig:ttau*)

</div>
<div id="div_fig3">

<img src="tmp_2401.15172/./figures/combined_acen_RL_alpha_dcz.png" alt="Fig11" width="100%"/>

**Figure 11. -** First row: Evolutionary tracks for \acen A (left panel) and \acen B (right panel) on the $R - L$ plane. Tracks computed in the present work are represented as solid lines, while the one from [ and Spada (2019)]() are drawn as dashed lines. Standard models are in blue and EC models in orange. Second row: Evolution of $\amlt$ as a function of age for the same models. Third row: Evolution of the convective envelope depth along evolution. (*fig:rl_alpha_dcz*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2401.15172"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# What stars can form planets: planetesimal formation at low metallicity

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2401.16155-b31b1b.svg)](https://arxiv.org/abs/2401.16155)<mark>Appeared on: 2024-01-30</mark> -  _Accepted for publication in A&A_

</div>
<div id="authors">

G. Andama, <mark>J. Mah</mark>, B. Bitsch

</div>
<div id="abstract">

**Abstract:** The exoplanet diversity has been linked to the disc environment in which they form, where the host star metallicity and the formation pathways play a crucial role. In the context of the core accretion paradigm, the initial stages of planet formation require the growth of dust material from micrometre size to planetesimal size bodies before core accretion can kick in. Although numerous studies have been conducted on planetesimal formation, it is still poorly understood how this process takes place in low metallicity stellar environments. In this work, we explore how planetesimals are formed in stellar environments primarily with low metallicity. We performed global 1D viscous disc evolution simulations including growth of dust particles,  evaporation and condensation of chemical species at ice lines. We followed the formation of planetesimals during  disc evolution and tested different metallicities, disc sizes and turbulent viscosity strengths.  We find that at solar and sub-solar metallicities, there is a significant enhancement in the midplane dust-to-gas mass ratios at the ice lines but this leads to planetesimal formation only at the water ice line. In our simulations, [ Fe/H ] $= -0.6$ is the lowest limit of metallicity for planetesimal formation where a few Earth masses of planetesimals could form. Our results further show that for such extreme disc environments, large discs are more conducive than small discs for forming large amounts of planetesimals at a fixed metallicity, because the pebble flux can be maintained for a longer time resulting in a longer and more efficient planetesimal formation phase. At lower metallicities, planetesimal formation  is less supported in quiescent discs compared to turbulent discs which produce relatively larger amounts of planetesimals, because the pebble flux can be maintained for a longer time. The amount of planetesimals formed at sub-solar metallicities in our simulations places a limit on core sizes that could possibly only result in the formation of super-Earths.

</div>

<div id="div_fig1">

<img src="tmp_2401.16155/./Figures/fig1_sigma_draz_rc100_alpha4.png" alt="Fig6" width="100%"/>

**Figure 6. -** _Top_: Evolution of pebble and planetesimal surface densities for a nominal $\alpha_{\rm t}=10^{-4}$ and disc size of $r_{\rm c}=100$ au and different values of disc metallicity. _Bottom_: The evolution of the Stokes numbers and the corresponding midplane dust-to-gas ratios. (*fig:fig1*)

</div>
<div id="div_fig2">

<img src="tmp_2401.16155/./Figures/fig2_sigma_draz_rc100_alpha3.png" alt="Fig7" width="100%"/>

**Figure 7. -** _Top_: Evolution of pebble and planetesimal surface densities for a nominal $\alpha_{\rm t}=10^{-3}$ and disc size of $r_{\rm c}=100$ au and different values of disc metallicity. _Bottom_: The evolution of the Stokes numbers and the corresponding midplane dust-to-gas ratios. (*fig:fig2*)

</div>
<div id="div_fig3">

<img src="tmp_2401.16155/./Figures/fig5_Mass_vs_t_rc_draz.png" alt="Fig4" width="100%"/>

**Figure 4. -** Disc size dependency of planetesimal formation at the water ice line. The vertical dotted lines have the same meaning as in Figure \ref{fig:fig4}. (*fig:fig5*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2401.16155"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

365  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
